In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Rekurencyjne sieci neuronowe



[...]

## Warstwa rekurencyjna w pakiecie Keras

Proces, który zaimplementowaliśmy przy użyciu biblioteki Numpy, jest odpowiednikiem prostej warstwy o nazwie SimpleRNN pakietu Keras:

In [2]:
from keras.layers import SimpleRNN

Pomiędzy tymi implementacjami jest jednak pewna mała różnica: warstwa SimpleRNN dzieli sekwencje na wsady (operację taką wykonują wszystkie warstwy Keras) — nie przetwarza pojedynczych sekwencji tak, jak to miało miejsce w przykładzie zaimplementowanym za pomocą biblioteki Numpy. W związku z tym warstwa ta przyjmuje obiekty wejściowe o kształcie (rozmiar_wsadu, kroki_czasu, cechy_wejściowe) a nie (kroki_czasu, cechy_wejściowe).

Warstwa SimpleRNN, podobnie jak wszystkie rekurencyjne warstwy pakietu Keras, może być uruchomiona w dwóch trybach: może zwracać pełne sekwencje kolejnych obiektów wyjściowych dla każdego kroku czasu (trójwymiarowe tensory o kształcie (rozmiar_wsadu, kroki_czasu, cechy_wyjściowe)) lub tylko ostatnie obiekty wyjściowe poszczególnych sekwencji wejściowych (dwuwymiarowe tensory o kształcie (rozmiar_wsadu, cechy_wyjściowe)). Wybór trybu pracy jest dokonywany za pomocą argumentu return_sequences. Przeanalizujmy przykład, w którym zastosowano warstwę SimpleRNN, a dane wyjściowe są zwracane tylko podczas przetwarzania ostatniego kroku czasu:

In [3]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [4]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


Czasami warto utworzyć stos składający się z kilku warstw rekurencyjnych. Zwiększa to siłę tworzenia reprezentacji przez sieć. Przy takiej konfiguracji wszystkie warstwy pośrednie muszą zwracać pełną sekwencję obiektów wyjściowych:

In [5]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))  # Ostatnia warstwa zwraca tylko ostatni obiekt wyjściowy.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 32)                2080      
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________


Teraz użyjmy takiego modelu w celu rozwiązania problemu klasyfikacji recenzji filmów. Zacznijmy od wstępnej obróbki danych:

In [6]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # Liczba słów traktowanych jako cechy.
maxlen = 500  # Ucina recenzje po tej liczbie słów należących do zbioru max_features słów najczęściej występujących w zbiorze.
batch_size = 32

print('Ładowanie danych...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'sekwencje treningowe')
print(len(input_test), 'sekwencje testowe')

print('Sekwencje (próbki x czas)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('Kształt obiektu input_train:', input_train.shape)
print('Kształt obiektu input_test:', input_test.shape)


Ładowanie danych...
17465344/17464789 [==============================] - 5s 0us/step
25000 sekwencje treningowe
25000 sekwencje testowe
Sekwencje (próbki x czas)
Kształt obiektu input_train: (25000, 500)
Kształt obiektu input_test: (25000, 500)


Przeprowadźmy proces trenowania prostej rekurencyjnej sieci przy użyciu warstwy Embedding i warstwy SimpleRNN.

In [ ]:
from keras.layers import Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 17s 828us/step - loss: 0.6453 - acc: 0.6114 - val_loss: 0.4864 - val_acc: 0.7936
Epoch 2/10
20000/20000 [==============================] - 16s 810us/step - loss: 0.4319 - acc: 0.8096 - val_loss: 0.5145 - val_acc: 0.7806
Epoch 3/10
20000/20000 [==============================] - 16s 811us/step - loss: 0.3169 - acc: 0.8700 - val_loss: 0.4348 - val_acc: 0.8042
Epoch 4/10
20000/20000 [==============================] - 16s 821us/step - loss: 0.2547 - acc: 0.8998 - val_loss: 0.4339 - val_acc: 0.7986
Epoch 5/10
20000/20000 [==============================] - 17s 853us/step - loss: 0.2089 - acc: 0.9222 - val_loss: 0.3891 - val_acc: 0.8642
Epoch 6/10
20000/20000 [==============================] - 17s 869us/step - loss: 0.1519 - acc: 0.9460 - val_loss: 0.3593 - val_acc: 0.8718
Epoch 7/10
20000/20000 [==============================] - 17s 842us/s

Teraz możemy wyświetlić wykresy dokładności i straty w procesach trenowania i walidacji:

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Dokladnosc trenowania')
plt.plot(epochs, val_acc, 'b', label='Dokladnosc walidacji')
plt.title('Dokladnosc trenowania i walidacji')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Strata trenowania')
plt.plot(epochs, val_loss, 'b', label='Strata walidacji')
plt.title('Strata trenowania i walidacji')
plt.legend()

plt.show()

Pierwsze maksymalnie uproszczone rozwiązanie, przedstawione w rozdziale 3., doprowadziło do uzyskania dokładności testowej na poziomie 88%. Niestety ta mała rekurencyjna sieć działa o wiele słabiej od wspomnianego rozwiązania (uzyskujemy dokładność walidacyjną na poziomie zaledwie 85%). Problem ten wynika częściowo z tego, że nie przetwarzamy pełnych sekwencji, a tylko 500 pierwszych słów, a więc sieć rekurencyjna ma dostęp do mniejszej ilości informacji niż model przedstawiony w rozdziale 3. Ponadto warstwa SimpleRNN nie sprawdza się najlepiej podczas przetwarzania długich sekwencji takich jak tekst. W takiej sytuacji lepiej sprawdzają się inne warstwy rekurencyjne. Przyjrzyjmy się bardziej zaawansowanym warstwom tego typu.

[...]

## Przykład warstwy LSTM zaimplementowanej w pakiecie Keras

Czas przyjrzeć się praktycznemu przykładowi zastosowania warstwy LSTM. Skonfigurujemy model, w którym znajdzie się taka warstwa, i wytrenujemy go na zbiorze danych IMDB (patrz rysunki 6.16 i 6.17). Przypomina on zaprezentowany wcześniej model z warstwą SimpleRNN. Określimy tylko liczbę wymiarów obiektu wyjściowego warstwy LSTM. Pozostałe argumenty tej warstwy (jest ich wiele) pozostawimy przy wartościach domyślnych. Ustawienia domyślne pakietu Keras są przemyślane i zwykle „po prostu działają” bez konieczności poświęcania dużej ilości czasu na ręczne dostrajanie parametrów.

In [ ]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Dokladnosc trenowania')
plt.plot(epochs, val_acc, 'b', label='Dokladnosc walidacji')
plt.title('Dokladnosc trenowania i walidacji')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Strata trenowania')
plt.plot(epochs, val_loss, 'b', label='Strata walidacji')
plt.title('Strata trenowania i walidacji')
plt.legend()

plt.show()